In [3]:
using BenchmarkTools
using NPZ

# Common

In [4]:
struct Point
    x::Float64
    y::Float64
end

# Direction vector
struct DVector
    x::Float64
    y::Float64
end

function cross_product(u::DVector, v::DVector)
    u.x * v.y - u.y * v.x
end

function dot_product(u::DVector, v::DVector)
    u.x * v.x + u.y * v.y
end

dot_product (generic function with 1 method)

# Sutherland-Hodgman

In [5]:
function _inside(p::Point, r::Point, U::DVector)
    U.x * (p.y - r.y) > U.y * (p.x - r.x)
end

function _intersection(a::Point, V::DVector, r::Point, N::DVector)
    W = DVector(r.x - a.x, r.y - a.y)
    nw = dot_product(N, W)
    nv = dot_product(N, V)
    if nv != 0.0
        t = nw / nv
        return true, Point(a.x + t * V.x, a.y + t * V.y)
    else
        return false, Point(0.0, 0.0)
    end
end

function polygon_area(polygon, len)
    area = 0.0
    a = Point(polygon[1, 1], polygon[2, 1])
    b = Point(polygon[1, 2], polygon[2, 2])
    U = DVector(b.x - a.x, b.y - a.y)
    for i in 3:len
        c = Point(polygon[1, i], polygon[2, i])
        V = DVector(a.x - c.x, a.y - c.y)
        area += abs(cross_product(U, V))
        b = c
        U = V
    end
    return 0.5 * area
end

function _copy(src, dst, n)
    for i in 1:n
        dst[1, i] = src[1, i]
        dst[2, i] = src[2, i]
    end
end

function _push(array, n, value)
    array[1, n + 1] = value.x
    array[2, n + 1] = value.y
    return n + 1
end

function clip_polygons(polygon, clipper)
    n_output = size(polygon)[2]
    n_clip = size(clipper)[2]
    output = Array{Float64}(undef, (2, 6))
    subject = Array{Float64}(undef, (2, 6))
    _copy(polygon, output, n_output)
    
    r = Point(clipper[1, n_clip], clipper[2, n_clip])
    for i in 1:n_clip
        s = Point(clipper[1, i], clipper[2, i])

        U = DVector(s.x - r.x, s.y - r.y)
        N = DVector(-U.y, U.x)
        if U.x == 0.0 && U.y == 0.0
            continue
        end
        len = n_output
        _copy(output, subject, len)
        n_output = 0

        a = Point(subject[1, len], subject[2, len])
        a_inside = _inside(a, r, U)
        for j in 1:len
            b = Point(subject[1, j], subject[2, j])

            V = DVector(b.x - a.x, b.y - a.y)
            if V.x == 0.0 && V.y == 0.0
                continue
            end
            b_inside = _inside(b, r, U)
            
            if b_inside
                if !a_inside
                    succes, point = _intersection(a, V, r, N)
                    if succes
                        n_output = _push(output, n_output, point)
                    end
                end
                n_output = _push(output, n_output, b)
            elseif a_inside
                succes, point = _intersection(a, V, r, N)
                if succes
                    n_output = _push(output, n_output, point)
                else
                    b_inside = true
                    n_output = _push(output, n_output, b)
                end
            end
            
            a = b
            a_inside = b_inside
        end
        
        if n_output < 3
            return 0.0
        end
        
        r = s
    end
    area = polygon_area(output, n_output)
    return area
end      

clip_polygons (generic function with 1 method)

# Read triangles

In [6]:
a = permutedims(npzread("triangles_a.npy"), (3, 2, 1))
b = permutedims(npzread("triangles_b.npy"), (3, 2, 1))

2×3×1000000 Array{Float64,3}:
[:, :, 1] =
 0.13943   0.20051   0.0433536
 0.141259  0.103664  0.345891 

[:, :, 2] =
 0.638903  0.859061  0.286762
 0.725644  0.911425  0.768978

[:, :, 3] =
 0.127354  0.668901  0.166028
 0.614252  0.977322  0.912566

...

[:, :, 999998] =
 0.163299  0.918414  0.731335
 0.846918  0.664982  0.812965

[:, :, 999999] =
 0.581572  0.0160035  0.949833
 0.862413  0.445922   0.728067

[:, :, 1000000] =
 0.932028  0.104979  0.533061
 0.607966  0.153133  0.193289

In [7]:
function area_of_intersection(a, b)
    n = size(a)[end]
    out = Array{Float64}(undef, n)
    for i in 1:n
        t0 = a[:, :, i]
        t1 = b[:, :, i]
        out[i] = clip_polygons(t0, t1)
    end
    return out
end

area_of_intersection (generic function with 1 method)

In [8]:
@btime area_of_intersection(a, b)

  311.246 ms (4000002 allocations: 587.46 MiB)


1000000-element Array{Float64,1}:
 0.0                  
 0.0                  
 0.0015428417818308345
 0.0116674946363678   
 0.0                  
 0.012820796709396648 
 0.015439009483970728 
 0.0011013956528890018
 0.01195865718660706  
 0.01127765391313546  
 0.0                  
 0.009645649033342713 
 0.03162643130242185  
 ⋮                    
 0.00952080431982822  
 0.0010082222361460235
 0.0                  
 0.01320632733056133  
 0.03502262207396337  
 0.0                  
 0.0                  
 0.0                  
 0.021154967845624113 
 5.482193249288619e-7 
 0.019657919361185133 
 0.01220281881976658  